In [1]:
# -*- coding: utf-8 -*-

# Jason Kahn
# Starting Better - 05/23/2024

# GENERAL DIRECTIONS FOR USE

# - FOR THE TIME BEING, YOU NEED MICROSOFT EXCEL TO RUN THIS PROGRAM
# - WHEN YOU RUN THIS FIRST PORTION, YOU WILL BE ASKED FOR NUMBER OF ROWS, COLUMNS, AND LAYERS 
# - AN EXCEL SPREADSHEET WILL OPEN, FORMATTED TO THE INPUTS YOU ENTERED. FILL IN THE SPREADSHEET ACCORDING TO YOUR DESIGN, 
#   WHERE THE INPUT MATERIAL DESIGN NEEDS TO BE CAPABLE OF TRANSLATION IN X,Y,Z SPACE WITH OVERLAPPING BOUNDARIES (SIMILAR TO A UNIT CELL)
# - DIFFERENT MATERIALS WILL BE INPUT USING 0,1,2,3,4...WHERE 0 IS AN EMPTY VOXEL AND 1 AND ABOVE ARE INTERNALLY-BOUND MATERIALS
#   THE (0,0,0) POSITION IS AT THE TOP LEFT CORNER OF LAYER 1 ON THE SPREADSHEET
# - SAVE THE EXCEL SPREADSHEET, AND CLOSE THE SPREADSHEET
# - ONCE THE SPREADHSEET IS CLOSED, COME BACK TO THE PROGRAM AND PRESS 'ENTER'
# - CONFIRM THE MATERIAL PLACEMENT YOU INPUT THROUGH EXCEL (USING 'Y' IF CORRECT AND 'N' FOR NOT CORRECT)
# - CLICK THE RUN BUTTON CONTINUOSLY TO GET THROUGH THE PROGRAM, TRACKING PROGRESS OF THE VOXEL REDUCTION AND BOND PAINTING
# - FINAL OUTPUT SHOULD RESPRESENT A MINIMAL SET OF VOXELS AND COLORS - THE PROGRAM ALLOWS FOR MULTIPLE BINDING OPTIONS FOR A GIVEN COLOR
#   IF THAT POSSIBILITY HAS NATURALLY ARISEN DUE TO SYMMETRY. THIS REPRESENTS A STARTING POINT FOR FUTURE ITERATIONS OF THE PROGRAM
#   THAT WILL BE CAPABLE OF PROVIDING MULTIPLE OUTPUTS, REDUCING THESE BINDING POSSIBILITIES BUT ALSO INCREASING THE NUMBER OF VOXELS AND COLORS
# - HOPEFULLY YOU WILL BE STUNNED AND AMAZED WHEN YOU SEE THE FINAL RESULTS (IN A GOOD WAY)
# - IF STUNNED OR AMAZED IN A BAD WAY, PLEASE NOTE ANY ERRORS
# - IF NEITHER, TEST THE PROGRAM FURTHER UNTIL YOU ARE ONE OF THE TWO
# - THE CODE IS IN NO WAY OPTIMIZED - THERE ARE SECTIONS CODING SIMILAR ASPECTS THAT ARE WRITTEN DIFFERENTLY, AND OFTEN
#   VARAIBLE DEFINITIONS ARE REPEATED. THE PROGRAM WAS WRITTEN IN STAGES, AND WILL BE FURTHER OPTIMIZED AND CORRECTED.
# - THANK YOU FOR ANY HELP AND FEEDBACK YOU PROVDIDE!



import numpy as np
import itertools
import xlsxwriter
import os
import xlrd
import sys
from vpython import *
import pandas as pd
import openpyxl as py
from pathlib import Path
from collections import OrderedDict
import copy


# DEFINE FUNCTION to draw borders for lattice input spreadsheet. Format cell borders via a configurable RxC box
# Function taken from the following website: https://stackoverflow.com/questions/21599809/python-xlsxwriter-set-border-around-multiple-cells
# Handles scenarios with single column or row
def draw_frame_border(workbook, worksheet, first_row, first_col, rows_count, cols_count,thickness=2):
        
            if cols_count == 1 and rows_count == 1:
                # whole cell
                worksheet.conditional_format(first_row, first_col,
                                             first_row, first_col,
                                             {'type': 'formula', 'criteria': 'True',
                                             'format': workbook.add_format({'top': thickness, 'bottom':thickness,
                                                                            'left': thickness,'right':thickness})})    
            elif rows_count == 1:
                # left cap
                worksheet.conditional_format(first_row, first_col,
                                         first_row, first_col,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'top': thickness, 'left': thickness,'bottom':thickness})})
                # top and bottom sides
                worksheet.conditional_format(first_row, first_col + 1,
                                         first_row, first_col + cols_count - 2,
                                         {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'top': thickness,'bottom':thickness})})
        
                # right cap
                worksheet.conditional_format(first_row, first_col+ cols_count - 1,
                                         first_row, first_col+ cols_count - 1,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'top': thickness, 'right': thickness,'bottom':thickness})})
        
            elif cols_count == 1:
                # top cap
                worksheet.conditional_format(first_row, first_col,
                                         first_row, first_col,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'top': thickness, 'left': thickness,'right':thickness})})
        
                # left and right sides
                worksheet.conditional_format(first_row + 1,              first_col,
                                         first_row + rows_count - 2, first_col,
                                         {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'left': thickness,'right':thickness})})
        
                # bottom cap
                worksheet.conditional_format(first_row + rows_count - 1, first_col,
                                         first_row + rows_count - 1, first_col,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'bottom': thickness, 'left': thickness,'right':thickness})})
        
            else:
                # top left corner
                worksheet.conditional_format(first_row, first_col,
                                         first_row, first_col,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'top': thickness, 'left': thickness})})
        
                # top right corner
                worksheet.conditional_format(first_row, first_col + cols_count - 1,
                                         first_row, first_col + cols_count - 1,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'top': thickness, 'right': thickness})})
        
                # bottom left corner
                worksheet.conditional_format(first_row + rows_count - 1, first_col,
                                         first_row + rows_count - 1, first_col,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'bottom': thickness, 'left': thickness})})
        
                # bottom right corner
                worksheet.conditional_format(first_row + rows_count - 1, first_col + cols_count - 1,
                                         first_row + rows_count - 1, first_col + cols_count - 1,
                                         {'type': 'formula', 'criteria': 'True',
                                          'format': workbook.add_format({'bottom': thickness, 'right': thickness})})
        
                # top
                worksheet.conditional_format(first_row, first_col + 1,
                                             first_row, first_col + cols_count - 2,
                                             {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'top': thickness})})
        
                # left
                worksheet.conditional_format(first_row + 1,              first_col,
                                             first_row + rows_count - 2, first_col,
                                             {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'left': thickness})})
        
                # bottom
                worksheet.conditional_format(first_row + rows_count - 1, first_col + 1,
                                             first_row + rows_count - 1, first_col + cols_count - 2,
                                             {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'bottom': thickness})})
        
                # right
                worksheet.conditional_format(first_row + 1,              first_col + cols_count - 1,
                                             first_row + rows_count - 2, first_col + cols_count - 1,
                                             {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'right': thickness})})

# Ask user for name of design - will be the title of file
DesignName= input('Please enter design name: ')                
# usertype defines how 'space' is built up from the motif - with tiling overlaps for a unit cell and no tiling overlaps for an arbitrary input                
askusertype = int(input("If you are inputting a unit cell, input 1, if inputting arbitrary unit cell, input 2. If inputting a unit cell, the mininum number of components in any dimension can be 3, and if inputting an arbitrary design, the minumum number can be 2."))
# BFlimit, or binding flexibility, determine what cutoff ratio in 'mapped' bonds between a voxel complement and its parent to assign new bond colors
BFlimit = int(input("What is binding flexibility bond limit. BOND LIMIT = "))
# number of rows in unit cell or motif
rows = int(input("Please enter the number of rows (x) in random lattice: NUMBER OF ROWS = "))  # x length
# number of columns in unit cell or motif
columns = int(input("Please enter the number of columns (y) in random lattice: NUMBER OF COLUMNS = "))  # y length
# number of layers in unit cell or motif
layers = int(input("Please enter the number of layers (z) in random lattice: NUMBER OF LAYERS = "))  # z length

# make copies of rows, columns, and layers because effective rows, columns, and layers will change based on motif or unit cell
orgrows = copy.deepcopy(rows)
orgcolumns = copy.deepcopy(columns)
orglayers = copy.deepcopy(layers)

# define input file name for user to input lattice
file = "InputParticleLattice.xlsx"
# string to start excel
string = "start EXCEL.EXE " + file

# Create Excel worksheet for user to input lattice organization
workbook = xlsxwriter.Workbook(file)
worksheet = workbook.add_worksheet()

 # Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': True})

# Define excel spreadsheet layout
worksheet.write(0, 0, 'PLEASE INPUT LATTICE INFORMATION ABOUT EITHER UNIT CELL OR LATTICE, WITH NUMBERS REPRESENTING EACH PARTICLE TYPE, EMPTY CELL = 0', bold)

for i in range(0,layers):

    stringwrite = "Please enter rows(x) and columns(y) for layer {}".format(i+1)
    looprow = 2+2*i+columns*i
    worksheet.write(looprow, 0, stringwrite)

#     draw_frame_border(workbook, worksheet, looprow+1, 0, rows, columns)
    draw_frame_border(workbook, worksheet, looprow+1, 0, columns, rows)

# Excel workbook program end
workbook.close()

# Open Excel file for user to input information, file must be in the same folder as program
directory = os.getcwd()
os.system(string)

# Wait for user fill in spreadsheet and save the file
input("Please fill out Excel spreedsheet, save the file, close the file, then press enter to continue")

# set aside numpy array filled with zeros to fill in with user input
partlattice = np.zeros((rows,columns,layers))
# create a secondary lattice that will output to the user the rows, columns, and layers in a more visual format.
# This array is not used in the program, just as a visual for the user to check
showpartlattice = np.zeros((layers,rows,columns))

# read file and fill in partlattice and showpartlattice numpy arrays
loc = (file)
wb = py.load_workbook(loc)
sheet = wb.active


for l in range(layers):
    for c in range(columns):
        for r in range(rows):
            looprow = 4 + (columns-1)*(l+1) +3*l #4+3*(l)+(rows-1)*l
            partlattice[r,c,l] = int(sheet.cell(column=r+1,row=looprow-c).value)
            showpartlattice[l,r,c] = partlattice[r,c,l]
            


# if userinput was a motif, not a unit cell, create an effective unit cell of this motif in another numpy array 'partlattice2'
# and then replace partlattice with this array
if askusertype == 2:
       
    rows += 1
    columns += 1
    layers += 1
    orgrows = rows - 1
    orgcolumns = columns - 1
    orglayers = layers - 1
    partlattice2 = np.zeros((rows,columns,layers))
    for i in range(rows):
        for j in range(columns):
            for k in range(layers):
                l = copy.deepcopy(i)
                m = copy.deepcopy(j)
                n = copy.deepcopy(k)
                if l == orgrows:
                    l = 0
                if m == orgcolumns:
                    m = 0
                if n == orglayers:
                    n = 0                
                partlattice2[i,j,k] = partlattice[l,m,n]

    partlattice = partlattice2

# show lattice input to user to check if information was put in correctly
print(showpartlattice)    
cont = input("Is particle array input correctly: Y or N = ")

<IPython.core.display.Javascript object>

Please enter design name: test
If you are inputting a unit cell, input 1, if inputting arbitrary unit cell, input 2. If inputting a unit cell, the mininum number of components in any dimension can be 3, and if inputting an arbitrary design, the minumum number can be 2.1
What is binding flexibility bond limit. BOND LIMIT = 2
Please enter the number of rows (x) in random lattice: NUMBER OF ROWS = 3
Please enter the number of columns (y) in random lattice: NUMBER OF COLUMNS = 4
Please enter the number of layers (z) in random lattice: NUMBER OF LAYERS = 6
Please fill out Excel spreedsheet, save the file, close the file, then press enter to continue
[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 0. 0.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]
Is particle array input correctly: Y or N = y


# Visualization functions and coloring 

In [2]:
#### Visualization functions using Vpython

# Define global color Library

# material coloring library
colordict = {0:color.white, 1:color.yellow, 2:color.red, 3:color.blue, 4:color.green}
binding_list = ["bc_pos_x", "bc_neg_x", "bc_pos_y", "bc_neg_y", "bc_pos_z", "bc_neg_z"]
# bond color library
extcolordict = {0:vector(1, 1, 1), 1:vector(0,1,1), 2:vector(0,1,0),3:vector(0.4,0.2,0.6), 4:vector(1,0,1),5:vector(1,0.6,0),
                6:vector(1,0,0), 7:vector(1,1,0), 8:vector(0,0,1),9:vector(0,0,0),10:vector(0,0.8,0),
                11:vector(0,0.5,0.5), 12:vector(0,0.5,0),13:vector(0.2,0.1,0.3), 14:vector(0.5,0,0.5),15:vector(0.5,0.3,0),
                16:vector(0.5,0,0), 17:vector(0.5,0.5,0), 18:vector(0,0,0.5),19:vector(0,0.4,0), 20:vector(0,.2,0), 21:vector(0,.2,.2),
                22:vector(.2,.2,.2), 23:vector(.2,.2,0), 24:vector(.2,0,.2), 25:vector(0,0,.2), 26:vector(.2,.4,.2), 27:vector(.2,.2,.4),
                28:vector(.4,.2,.2), 29:vector(.2,.4,.4), 30:vector(.4,.4,.2), 31:vector(.4,.2,.4), 32:vector(.4,.4,.4), 33:vector(.4,.4,.6),
                34:vector(.4,.6,.4), 35:vector(.6,.4,.4), 36:vector(.4,.6,.6), 37:vector(.6,.6,.4), 38:vector(.6,.4,.6), 39:vector(.6,.6,.6),
                40:vector(.6,.6,.8), 41:vector(.6,.8,.6), 42:vector(.8,.6,.6), 43:vector(.6,.8,.8), 44:vector(.8,.6,.8), 45:vector(.8,.8,.8),
                46:vector(.8,.8,1), 47:vector(.8,1,.8), 48:vector(1,.8,.8), 49:vector(.8,1,1), 50:vector(1,1,.8),51:vector(1,1,.8),
                51:vector(0.3,0.3,0.3), 52:vector(0.3,0.3,0.5),53:vector(0.3,0.5,0.3),54:vector(0.5,0.3,0.3),55:vector(0.5,0.5,0.3),
                56:vector(0.3,0.5,0.5),57:vector(0.5,0.3,0.5),58:vector(0.5,0.5,0.5),59:vector(0.5,0.5,0.7),60:vector(0.5,0.7,0.5),
                61:vector(0.7,0.5,0.5),62:vector(0.5,0.7,0.7),63:vector(0.7,0.5,0.7),64:vector(0.7,0.7,0.5),65:vector(0.7,0.7,0.7),
                66:vector(0.7,0.7,0.9),67:vector(0.7,0.9,0.7),68:vector(0.9,0.7,0.7),69:vector(0.7,0.9,0.9),70:vector(0.9,0.7,0.9),
                71:vector(0.9,0.9,0.7),72:vector(0.9,0.9,0.9)}

# Function that adds axes to Vpython scene
def axes():
    pointerx = arrow(pos=vector(-8,-5,-4),      axis=vector(5,0,0), shaftwidth=0, color = vector(1,0,0))
    text(text='x-axis', pos=vector(-6,-4,-4),align='upper_right', color=vector(1,0,0), axis=vector(1,0,0))
    pointery = arrow(pos=vector(-8,-5,-4),      axis=vector(0,5,0), shaftwidth=0, color = vector(0,1,0))
    text(text='y-axis', pos=vector(-9,0,-4),align='upper_right', color=vector(0,1,0), axis=vector(1,0,0))
    pointerz = arrow(pos=vector(-8,-5,-4),      axis=vector(0,0,5), shaftwidth=0, color = vector(0,0,1))
    text(text='z-axis', pos=vector(-9,-7,0),align='upper_right', color=vector(0,0,1), axis=vector(1,0,0))

# Showcolors function outputs Vpython scene from voxel objects, not Numpy array
def showcolors(mesovoxel):
    
    scene = canvas(center=vector(0,0,0),
                        title='3D Graphics by VPython', width=800,height=600,
                        background= colordict[0])
    # Set vectors for binding directions and length of visual bond
    vectors = tuple([vector(2,0,0), vector(-2,0,0), vector(0,2,0), vector(0,-2,0), vector(0,0,2), vector(0,0,-2)])
    
    # Add axes
    axes()

    # Cycle through all user input voxels
    for obj in mesovoxel:   
        [a,b,c] = [obj.coordinates[0],obj.coordinates[1],obj.coordinates[2]]
        if a == orgrows or b == orgcolumns or c == orglayers:
            continue
     
        color1 = colordict[obj.material] #color of the voxel's internal particle
        
        # First checks for the complementarity type - 1 is rod, 2 is arrow (0 is default rod)
        # Colors are not affected by complementarity, as in there can be same color rods and arrows

        for i in range(6): # Cycle through binding directions
            bind_color = getattr(obj,binding_list[i])[0] # Call binding_list cooor attribute from voxel object
            bind_type = getattr(obj,binding_list[i])[1] # Call binding_list type attribute from voxel object
            if bind_type == 1: #if complementarity = 1, it gives cylinder object, with the corresponding binding color
                cylinder(axis=vectors[i], radius=0.3, color=extcolordict[bind_color], pos=vector(5*obj.coordinates[0],5*obj.coordinates[1], 5*obj.coordinates[2]))
            elif bind_type == 2: #if complementarity = 2, it gives arrow object, with the corresponding binding color
                arrow(axis=vectors[i], length=2.8, color=extcolordict[bind_color],pos=vector(5*obj.coordinates[0],5*obj.coordinates[1], 5*obj.coordinates[2]))
            else: # this means empty there is no complementarity, by default it is cylinder with white color, which means empty
                cylinder(axis=vectors[i], radius=0.3, color=extcolordict[bind_color], pos=vector(5*obj.coordinates[0],5*obj.coordinates[1], 5*obj.coordinates[2]))

        # Color internal particles or leave empty if no particle present  
        if obj.material == 0: #empty voxel
            sphere(color=color1, opacity = 0, radius =1, 
                   pos=vector(5*obj.coordinates[0],5*obj.coordinates[1], 5*obj.coordinates[2])) #empty
        else: #it means colors rather than 0, which is not empty
            sphere(color=color1, radius =1,pos=vector(5*obj.coordinates[0],
                                                      5*obj.coordinates[1], 5*obj.coordinates[2]))

            
            

# Building out effective 3D environment of input motif

In [3]:
# This builds out unit cell to at least one repeat in each direction, but due to later symmetry operations, need
# to account for different rotations and thus takes into the maximum dimension to build out potentially
# non-uniformally depending on dimensions of unit cell (hence the while loops compared to maximum dimension)


maxdim = max(rows, columns, layers)         
repeatrow = 3
repeatcol = 3
repeatlay = 3

fullrows = rows*repeatrow-(repeatrow-1)
addedrows = (repeatrow-1)*(rows-1)
while addedrows < (2*maxdim-2):
    repeatrow += 1
    fullrows = rows*repeatrow-(repeatrow-1)
    addedrows = (repeatrow-1)*(rows-1)

if (repeatrow % 2) == 0:
    repeatrow += 1
    fullrows += rows - 1

fullcolumns = columns*repeatcol-(repeatcol-1)
addedcols = (repeatcol-1)*(columns-1)
while addedcols < (2*maxdim-2):
    repeatcol += 1
    fullcolumns = columns*repeatcol-(repeatcol-1)
    addedcols = (repeatcol-1)*(columns-1)

if (repeatcol % 2) == 0:
    repeatcol += 1
    fullcolumns += columns - 1


fulllayers = layers*repeatlay-(repeatlay-1)
addedlays = (repeatlay-1)*(layers-1)
while addedlays < (2*maxdim-2):       
    repeatlay += 1
    fulllayers = layers*repeatlay-(repeatlay-1)
    addedlays = (repeatlay-1)*(layers-1)

if (repeatlay % 2) == 0:
    repeatlay += 1
    fullrows += columns - 1


# Copy over partlattice array to Numpy array
# testarray = np.array(partlattice)
# Create array of zeros with dimension the full size of 3D build out
fulltestarray = np.zeros((fullrows,fullcolumns,fulllayers))
# Build out 3D space
build = np.vstack((partlattice,partlattice[1:rows,:,:]))
build = np.hstack((build,build[:,1:columns,:]))
build = np.dstack((build,build[:,:,1:layers]))

# Build out 3D lattice
if repeatrow > 2:
    additional = repeatrow - 2
    for i in range(0,additional):
        build = np.vstack((build,build[1:rows,:,:]))

if repeatcol > 2:
    additional = repeatcol - 2
    for i in range(0,additional):           
        build = np.hstack((build,build[:,1:columns,:]))                 

if repeatlay > 2:
    additional = repeatlay - 2
    for i in range(0,additional):
        build = np.dstack((build,build[:,:,1:layers]))  

fulltestarray = build


In [4]:
partlattice

array([[[1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.]]])

#####  Define Voxel Class and fill in voxel list for every voxel in user input

In [5]:
### Define class for Voxels
class Voxel:
    def __init__(self, number, material, coordinates, bp_pos_x, bp_neg_x, bp_pos_y,bp_neg_y,bp_pos_z,bp_neg_z,
                 bc_pos_x, bc_neg_x, bc_pos_y,bc_neg_y,bc_pos_z,bc_neg_z, binding_list, surround_matrix, sym_vox, sym_index, uniqueness):
        '''Each of the voxels should contain the following information handles:
                Voxel number
                Material/Particle Type
                Particle coordinates in the testarray matrix (axbxc)
                +x neighbor
                -x neighbor
                +y neighbor
                -y neighbor
                +z neighbor
                -z neighbor
                +x binding [binding color, complementarity (1 for sequence or 2 for sequence complement)]
                -x biding
                +y binding
                -y binding
                +z binding
                -z binding
                "Binding List" - is a collection of all of the binding paramaters, including binding color and complementarity
                "surround_matrix" - This matrix is created by using the ‘fulltestarray’ matrix, which is the unit cell built out in 3D space. 
                using the voxel location, the surroundings matrix has x dimensions of +/- (a-1), 
                y dimensions of +/- (b-1), and z dimensions of +/- (c-1).
                symmetry - list of voxels the current voxels possesses symmetry with, ordered as [Symmetry voxel,symmetry element(s)]
                "sym_self" - s this is rotations and reflections
                "sym_set" - all the voxels that the current voxel has symmetry with
                "sym_dict - connects the symmetry voxels with the first shared symmetry element in order of translation, transofmration, double transofmration
                "neighbors" - all binding partners in order +x, -x, +y, -y, +z, -z
                "bond_change_dict" - used at end of program when determing whether unpainted voxels map to structural or complementary voxel
                "uniquness" - [uniqueness classifer (1 for structural voxel, 2 for required complement, 3 for otherwise), sym voxel, sym operation]'''
        
                
                
        
        # Fill each materials characteristics:
        self.number = number
        self.material = material #voxel's material, particle type 0: empty, 1,2,3 etc
        self.coordinates = coordinates #voxel's coordinates in the unit cell
        # These are placeholders containing the binding partner neighbor, will be filled in after voxels defined
        self.bp_pos_x = bp_pos_x # binding partner in positive x-direction, 0: empty, 1,2,3 etc
        self.bp_neg_x = bp_neg_x # binding partner in negative x-direction, 0: empty, 1,2,3 etc
        self.bp_pos_y = bp_pos_y # binding partner in positive y-direction, 0: empty, 1,2,3 etc
        self.bp_neg_y = bp_neg_y # binding partner in negative y-direction, 0: empty, 1,2,3 etc
        self.bp_pos_z = bp_pos_z # binding partner in positive z-direction, 0: empty, 1,2,3 etc
        self.bp_neg_z = bp_neg_z # binding partner in negative z-direction, 0: empty, 1,2,3 etc
        self.neighbors = [bp_pos_x, bp_neg_x, bp_pos_y, bp_neg_y, bp_pos_z, bp_neg_z]
        
        # These are binding colors, which contains two values
        #[0,0]: means empty, [1,1],[1,2]: first number means color type, second number means complementarity
        #if the first number is the same and second numbers: 1 and 2 means they are complementary.
        #first number can be any natural number, while second number can be either 0 or 1 and 2
        
        self.bc_pos_x = bc_pos_x #binding color in positive x-direction, 
        self.bc_neg_x = bc_neg_x #binding color in negative x-direction
        self.bc_pos_y = bc_pos_y #binding color in positive y-direction
        self.bc_neg_y = bc_neg_y #binding color in negative y-direction
        self.bc_pos_z = bc_pos_z #binding color in positive z-direction
        self.bc_neg_z = bc_neg_z #binding color in negative z-direction
        self.binding_list = [bc_pos_x, bc_neg_x, bc_pos_y, bc_neg_y, bc_pos_z, bc_neg_z] # list of binding attributes
        self.surround_matrix = surround_matrix #surroundings matrix for the voxel
        self.sym_voxels = []
        self.sym_self = [[]]
        self.uniqueness = uniqueness #uniqueness of the voxel according to the surroundings matrix
        self.sym_set = set()
        self.sym_dict = dict()
        self.bond_change_dict = dict()
        
    # give the full information about the voxel
    def displayVoxel(self):
        '''This  function displays the atributes of each voxel'''
        print("Material : ", self.material,  ", Coordinates: ", self.coordinates, ", binding pattern in posx: ",self.bp_pos_x, ", binding pattern in negx: ",self.bp_neg_x,
              ", binding pattern in posy: ",self.bp_pos_y, ", binding pattern in negy: ",self.bp_neg_y,", binding pattern in posz: ",self.bp_pos_z, ", binding pattern in negz: ",self.bp_neg_z,
              ", binding color in posx: ",self.bc_pos_x, ", binding color in negx: ",self.bc_neg_x,
              ", binding color in posy: ",self.bc_pos_y, ", binding color in negy: ",self.bc_neg_y,", binding color in posz: ",self.bc_pos_z, ", binding color in negz: ",self.bc_neg_z,
              ", surrounding matrix: ",self.surround_matrix, ", uniqueness: ",self.uniqueness)


In [6]:
# Gives the total number of the voxels in the input unit cell. This will be used later so is simply being filled now
voxel_count = []
for name in range(0,partlattice.size): # Make list of numbers up to total voxel number
    voxel_count.append(name)
print('In total, we have ', partlattice.size, ' voxels')


indexes = []
materials = []
for index, material in np.ndenumerate(partlattice):
    indexes.append(index)  #coordinates of the voxel in the unit cell
    materials.append(material) #particle in each voxel

#to avoid the boundary issues while finding the neighbor voxels, [m,p,o] is defined


m = int(floor(repeatrow/2)*(rows-1))

p = int(floor(repeatcol/2)*(columns-1))

o = int(floor(repeatlay/2)*(layers-1))
    

# Collect binding partners for each voxel in all directions
bp_posx = [] #collects the binding partners in pos x-direction
bp_negx = [] #collects the binding partners in neg x-direction
bp_posy = [] #collects the binding partners in pos y-direction
bp_negy = [] #collects the binding partners in neg y-direction
bp_posz = [] #collects the binding partners in pos z-direction
bp_negz = [] #collects the binding partners in neg z-direction
   
# Initialize binding colors for each voxel in eah direction    
bc_posx = [] # Collect the binding partners in pos x-direction
bc_negx = [] # Collect the binding partners in neg x-direction
bc_posy = [] # Collect the binding partners in pos y-direction
bc_negy = [] # Collect the binding partners in neg y-direction
bc_posz = [] # Collect the binding partners in pos z-direction
bc_negz = [] # Collect the binding partners in neg z-direction

for n in range(0,partlattice.size):
    bc_posx.append([0,0]) #by default, all the binding colors are [0,0], empty color, w/0 complementarity
    bc_negx.append([0,0]) #by default, all the binding colors are [0,0], empty color, w/0 complementarity
    bc_posy.append([0,0]) #by default, all the binding colors are [0,0], empty color, w/0 complementarity
    bc_negy.append([0,0]) #by default, all the binding colors are [0,0], empty color, w/0 complementarity
    bc_posz.append([0,0]) #by default, all the binding colors are [0,0], empty color, w/0 complementarity
    bc_negz.append([0,0]) #by default, all the binding colors are [0,0], empty color, w/0 complementarity    

# Collect the surroundings matrix for each voxel
surroundings = []

for i in range(0,partlattice.size):
    x_0 = m + indexes[i][0]
    y_0 = p + indexes[i][1]
    z_0 = o + indexes[i][2]

    surr_mat = fulltestarray[(x_0-(maxdim-1)):(x_0+(maxdim-1))+1,(y_0-(maxdim-1)):(y_0+(maxdim-1))+1,(z_0-(maxdim-1)):(z_0+(maxdim-1))+1]
    surroundings.append(surr_mat)  # gives the surroundings matrix has x dimensions of +/- (a-1), y dimensions of +/- (b-1), and z dimensions of +/- (c-1).

    
uniq_test_0 = []
for x in voxel_count:
    uniq_test_0.append([10,10,10])

sym_voxels_list = []
for x in voxel_count:
    sym_voxels_list.append([])

sym_index_list = []
for x in voxel_count:
    sym_index_list.append([])
    
#to define the attribues in each Voxel object
voxels = []

for i in range(0,partlattice.size):
    voxels.append(Voxel(i,materials[i],indexes[i],[], [],
                        [], [],
                        [], [],
                        bc_posx[i], bc_negx[i], bc_posy[i], 
                        bc_negy[i],bc_posz[i], bc_negz[i], [bc_posx[i], bc_negx[i], bc_posy[i], bc_negy[i],bc_posz[i], bc_negz[i]],
                        surroundings[i],sym_voxels_list[i], sym_index_list[i], uniq_test_0[i]))
        

In total, we have  72  voxels


In [7]:
partlattice

array([[[1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.]],

       [[1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0., 1.]]])

In [8]:
def refresh_binding(): # Function to update voxel binding_list attribute from the separate directional binding attributes
    for i in voxels:
        temp = []
        for j in range(6):
            temp.append(getattr(i,bond_traits[j]))

        i.binding_list = temp
    return

# Find binding partners according to assigned voxel numbers 

In [9]:
# This section fills in the binding partner attributes for all voxels based on the assigned voxel numbers and
# the direction of the binding

binddir = [(1,0,0), (-1,0,0),(0,1,0),(0,-1,0),(0,0,1),(0,0,-1)]
bond_partners = ["bp_pos_x", "bp_neg_x", "bp_pos_y", "bp_neg_y", "bp_pos_z", "bp_neg_z"]

def findpartner(listcoord):
    coordinates = tuple(listcoord)
    #print(coordinates)
    for bp in voxels:
        if bp.coordinates == coordinates:
            partner = bp.number 
    return partner

def neighbors(modneighb): # modneighb determines whether last row, column, layer is repeat of first, input 0 is unit cell repeat, input 1 is not
    for obj in voxels:
        setattr(obj,'neighbors',[0,0,0,0,0,0])
        for i in range(6):
            setattr(obj,bond_partners[i],[0])
    for obj in voxels:
        for i in range(6):
            
            listcoord = list(sum(x) for x in zip(obj.coordinates,binddir[i]))   
            if i == 0:
                if listcoord[0] == rows and modneighb == 0:
                    #print('This is xxxxxx', listcoord[0])
                    listcoord[0] = 0
                    partner = findpartner(listcoord)
                    obj.neighbors = voxels[partner].neighbors
                    #skip = 1
                    break
                elif listcoord[0] == rows and modneighb == 1:
                    listcoord[0] = 1
                elif listcoord[0] == (rows - 1) and modneighb == 0:
                    listcoord[0] = 0
                partner = findpartner(listcoord)                
                obj.neighbors[i] = partner

            if i == 1:
                if listcoord[0] == -1:
                    listcoord[0] = (rows - 2)
                partner = findpartner(listcoord)
                obj.bp_neg_x = partner
                obj.neighbors[i] = partner

            if i == 2:
                if listcoord[1] == columns and modneighb == 0:
                    listcoord[1] = 0
                    partner = findpartner(listcoord)
                    obj.neighbors = voxels[partner].neighbors
                    break
                elif listcoord[1] == columns and modneighb == 1:
                    listcoord[1] = 1
                elif listcoord[1] == (columns - 1) and modneighb == 0:
                    listcoord[1] = 0
                partner = findpartner(listcoord)
                obj.neighbors[i] = partner

            if i == 3: 
                if listcoord[1] == -1:             
                    listcoord[1] = (columns - 2)
                partner = findpartner(listcoord)
                obj.bp_neg_y = partner
                obj.neighbors[i] = partner

            if i == 4:
                if listcoord[2] == layers and modneighb == 0:
                    listcoord[2] = 0
                    partner = findpartner(listcoord)
                    obj.neighbors = voxels[partner].neighbors
                    break
                elif listcoord[2] == layers and modneighb == 1:
                    listcoord[2] = 1
                elif listcoord[2] == (layers - 1)  and modneighb == 0:
                    listcoord[2] = 0
                partner = findpartner(listcoord)
                obj.neighbors[i] = partner


            if i == 5:
                if listcoord[2] == -1:
                    listcoord[2] = (layers - 2)
                partner = findpartner(listcoord)
                obj.bp_neg_z = partner
                obj.neighbors[i] = partner
         
    for obj in voxels:
        for i in range(6):
            setattr(obj,bond_partners[i],obj.neighbors[i])
                
    return

In [10]:
# Run neighbors function
neighbors(0)   # Sends with neighbor 0

In [11]:
voxels[10].material

0.0

# User input with bonds showing for all voxels

In [12]:
# show the external colors of the voxel
# initially all the external colors are empty [0,0]
showcolors(voxels)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Entire material environment that will used by program based on user input

In [28]:
scene = canvas(center=vector(0,0,0),
                            title='3D Graphics by VPython', width=800,height=600,
                            background= colordict[0])
# visualize axes
axes()

# Create visual output
for i in range(0,fullrows):
    for j in range(0,fullcolumns):
        for z in range(0,fulllayers):
            if fulltestarray[i,j,z] >= 1: # This is for the actual particles
                sphere(pos=vector(2*i,2*j,2*z),color=colordict[fulltestarray[i,j,z]], radius =.5, make_trail=False, retain=50, center=(5,0,0), background=(1,1,1))
            else: # This is for the empty locations, change opacity to represent 'empty'
                sphere(pos=vector(2*i,2*j,2*z),color=colordict[fulltestarray[i,j,z]], opacity = 0, radius =.5, make_trail=False, retain=50, center=(5,0,0), background=(1,1,1))

<IPython.core.display.Javascript object>

## Reduced number of voxels (via surrounding matrix only)

In [30]:
mesovoxel = []
for i in voxels:
    mesovoxel.append(i)

# Add translation symmetry elements to "sym_voxels" attribute for each voxel object
for i in voxels:
    for j in voxels:
        if i != j and (i.surround_matrix == j.surround_matrix).all():
            i.sym_voxels.append([j.number,1000])

# remove voxels that share translational symmetry, going up in voxel number values, and add uniqueness identifiers to voxels
for i in mesovoxel:
    setattr(i,'uniqueness',[1,0,0])
    temp = i.sym_voxels
    for j in temp:
        mesovoxel.remove(voxels[j[0]])
        setattr(voxels[j[0]],'uniqueness',[3,i.number,1000])   

ValueError: list.remove(x): x not in list

# User input after voxels removed through translational symmetry

In [31]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

In [32]:
unique_voxels_index = []
for i in mesovoxel:
    unique_voxels_index.append(i.number)
print('The list of unique voxels after translation comparisons is', unique_voxels_index)

The list of unique voxels after translation comparisons is [0, 1, 2, 3, 4, 7, 8, 9, 10, 13, 14, 15, 16, 19, 20, 21, 22, 25, 26, 27, 28, 31, 32, 33, 34, 37, 38, 39, 40, 43, 44, 45, 46, 49, 50, 51, 52, 55, 56, 57, 58, 61, 62, 63, 64, 67, 68, 69, 70]


In [33]:
voxels[0].surround

AttributeError: 'Voxel' object has no attribute 'surround'

## Symmetry Transformations

After reducing the voxel number by solely comparing the surrounding matrix, 12 symmetry operations are applied to each of the voxels in the reduced voxel set. A list of symmetry transformations is prepared by comparing the transformed surrounding matrix of the voxel with the other voxels' original surrounding matrix. Only the 9 rotation operations are utilized in the rest of the program, but the others are listed as additional information and for additional functionality in the future.

In [34]:
global dbl
dbl = 0

def symmetry_results(unique_voxels_index):
    global trans_functions
    #print('double =', dbl)
    #def symmetry_results(unique_voxels_index):
    red_surr_mat = [] #it collects the surrounding matrices for the unique voxels
    

    
   
    for i in unique_voxels_index:
        red_surr_mat.append(voxels[i].surround_matrix)   # collects surroundings matrix for the reduced voxel set  

    # Create transformation list that defines the transformation functions and stores transformation matrices for each voxel
    # The list has the following order: ['rotation_90_x', 'rotation_180_x', 'rotation_270_x', 'rotation_90_y', 'rotation_180_y', 'rotation_270_y',
    #                                    'rotation_90_z', 'rotation_180_z', 'rotation_270_z', 'mirror_x', 'mirror_y', 'mirror_z']
   
    trans_functions =     [[[],np.rot90,1,(1,2)], [[],np.rot90,2,(1,2)], [[],np.rot90,3,(1,2)],
                          [[],np.rot90,1,(2,0)], [[],np.rot90,2,(2,0)], [[],np.rot90,3,(2,0)],
                          [[],np.rot90,1,(0,1)], [[],np.rot90,2,(0,1)], [[],np.rot90,3,(0,1)],
                          [[],np.flip,(0)],      [[],np.flip,(1)],      [[],np.flip,(2)]]

    snames = ['90° Rotation in x axis', '180° Rotation in x axis', '270° Rotation in x axis', '90° Rotation in y axis', '180° Rotation in y axis', '270° Rotation in y axis',
                     '90° Rotation in z axis', '180° Rotation in z axis', '270° Rotation in z axis', 'Reflection in x-plane', 'Reflection in y-plane', 'Reflection in z-plane']                              
    symmetry_lists = [[],[],[],[],[],[],[],[],[],[],[],[]]
    symmetry_limit = 9

        # Fill trans_functions with transformation matrices of each voxel after the stated rotation
    for x in range(len(trans_functions)):
        for i in range(len(unique_voxels_index)):
            
            
            
            if x<symmetry_limit: # rotations
                # np.rot90(matrix,numberrotations,axis)
                trans_functions[x][0].append(trans_functions[x][1](red_surr_mat[i], trans_functions[x][2], axes=trans_functions[x][3])) #determine the 90 degree rotation to the surrounding matrix of the given voxel
            else: # reflections
                trans_functions[x][0].append(trans_functions[x][1](red_surr_mat[i], trans_functions[x][2])) 


    for i in range(len(trans_functions)):    # cycle through all possible transformations
        #for x in trans_functions[i][0]:      # goes through each voxel in a given transformation matrix
        for s in range(len(unique_voxels_index)):
            x = trans_functions[i][0][s]           
            grouping = []
            for j in range(len(unique_voxels_index)):
                if x.shape != red_surr_mat[j].shape: # just a check for matrix size comparison - should be equal
                    grouping.append([])
                elif(x == red_surr_mat[j]).all():
                    grouping.append([0]) # if same assign 0                  
                else:
                    grouping.append([1]) # if not same asign 1


            x2 = np.array(grouping)  # make array from list

            findequal = []
            for n in np.where(x2 == 0)[0]:  # find the values with 0
                findequal.append((unique_voxels_index[n])) # append voxels associated with these 0 values (meaning the same)
            symmetry_lists[i].append(findequal)

    # Create dictionary for rotation symmetry matrices from predefined list of names and empty symmetry dictionary
    symmetry = {'Voxel Number': unique_voxels_index}
    for x in range(len(snames)):
        symmetry[snames[x]] = symmetry_lists[x] 

    #dataframe of symmetry operations
    df = pd.DataFrame(symmetry)

    # This section creates a list, called 'full_double_symmetry_list', that holds double rotations for every voxel. There are
    # no repeat symmetry transformation - for example, there are 12 transformations total (9 rotations, 3 reflections)
    # To get full double rotation list, chekc that function input is the whole voxel set
    
    if len(unique_voxels_index) == len(voxels):
        
        full_double_symmetry_list = [[0,0,0,0,0,0,0,0,0,0,0,0] for i in range(len(voxels))]
             
        for i in range(len(voxels)):
             # blank list to hold matrices after voxel's symmetry operation
            
            for j in range(len(trans_functions)):
                temp_symmetry_list = [0,0,0,0,0,0,0,0,0,0,0,0]
                first_matrix = trans_functions[j][0][i]   
               
                for k in range(j,len(trans_functions)):
                    if k < 9: # rotation functions
                        second_matrix = trans_functions[k][1](first_matrix, trans_functions[k][2], axes=trans_functions[k][3])
                    else: # reflection functions
                        second_matrix = trans_functions[k][1](first_matrix, trans_functions[k][2])

                    temp_symmetry_list[k] = second_matrix
                   
                    full_double_symmetry_list[i][j] = temp_symmetry_list  
                   
    if dbl == 0:
        return df
    elif dbl == 1:
        return  full_double_symmetry_list

In [35]:
dbl = 0
df = symmetry_results(unique_voxels_index)
# df

## Minimum number of voxels (via symmetry operations)

In [36]:
# # convert from dataframe to list values to make it easy to iterate
# symmetry_database = []
# for i in unique_voxels_index:
#     for index, row in df.loc[df['Voxel Number'] == i].iterrows():
#         # go through symmetry database and create lists for each row
        
#         symmetry_database.append([[row['Voxel Number']], row['90° Rotation in x axis'], row['180° Rotation in x axis'], 
#               row['270° Rotation in x axis'], row['90° Rotation in y axis'], row['180° Rotation in y axis'],
#               row['270° Rotation in y axis'], row['90° Rotation in z axis'], row['180° Rotation in z axis'],
#               row['270° Rotation in z axis'], row['Reflection in x-plane'], row['Reflection in y-plane'],
#               row['Reflection in z-plane']])

# similar_by = []
# for m in range(len(symmetry_database)):
#     similar_by.append([x for x in symmetry_database[m] if x]) # takes values if there is a value present


# # find the unique voxel indexes
# uniq_min = []
# for m in range(len(similar_by)):
#     x = np.array(similar_by[m])
#     uniq_min.append(np.unique(x))

# #minimized set of voxel numbers after the transformations (removing the not unique ones)
# min_voxels_index = []
# remove_list = []
# # This section takes the lowest numbered voxel for each row, keeps it in temp_list, and places other voxel numbers in bad_list if they are that number's symmetry list
# for i in uniq_min:
#     if i[0] not in min_voxels_index and i[0] not in remove_list: # check if the lowest number in that symmetry list is not already in bad_list
#         min_voxels_index.append(i[0])
#         if len(i)>1:
#             for j in range(len(i)-1):               
#                 remove_list.append(i[j+1])
                    


# # mesovoxel = [] # empty list to collect voxel objects in the minimum cell after symmetry transformations

# # for i in min_voxels_index:
# #     mesovoxel.append(voxels[i])  # Fill minimum central cell list
    
# print('The minimized list of voxels is now ', min_voxels_index)
# save_min_bef_dbrot = min_voxels_index[:]

# After single rotations used to remove voxels

In [37]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

# Symmetry list for all voxels

In [38]:
full_symmetry_list = symmetry_results(voxel_count)

In [39]:
# Create double rotation list
dbl = 1
double_symmetry_list = np.array(symmetry_results(voxel_count))

C:\Users\jkahn\AppData\Local\Temp\1/ipykernel_26916/318395079.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  double_symmetry_list = np.array(symmetry_results(voxel_count))


# Create conditional array for each voxel that corresponds to rotations list

In [40]:
# This section is similar to section reducing number of voxels, but without the similariy section needed
# Here, the full symmetry list for all voxels needs to be compared to create master symmetry list for voxel bond painting
# and complementarity function arising later in code

full_symmetry_database = []
for i in voxel_count:
    for index, row in full_symmetry_list.loc[full_symmetry_list['Voxel Number'] == i].iterrows():

        full_symmetry_database.append([[row['Voxel Number']], row['90° Rotation in x axis'], row['180° Rotation in x axis'], 
              row['270° Rotation in x axis'], row['90° Rotation in y axis'], row['180° Rotation in y axis'],
              row['270° Rotation in y axis'], row['90° Rotation in z axis'], row['180° Rotation in z axis'],
              row['270° Rotation in z axis'], row['Reflection in x-plane'], row['Reflection in y-plane'],
              row['Reflection in z-plane']])
        symmetry_limit = 12

for i in range(len(full_symmetry_database)):
    for j in range(len(full_symmetry_database[0])):
        if full_symmetry_database[i][j] == []:
            full_symmetry_database[i][j] = ["-"]

# Just gets the list numbering of voxels
similar_byfull1 = [i[0] for i in full_symmetry_database]
# For each voxel, make a list of other voxels it has rotionatal or reflection symmetry with
similar_byfull2 = [i[1:13] for i in full_symmetry_database]

# Adds symmetry list to voxel object
for i in range(len(similar_byfull2)): # This should be the same length as total voxels, or the list voxels
    temp1 = [] 
    for j in range(len(similar_byfull2[0])):
        for k in range(len(similar_byfull2[i][j])):
            if similar_byfull2[i][j][k] != i and similar_byfull2[i][j][k] != '-':           
                addlist = [similar_byfull2[i][j][k],j] 
                voxels[i].sym_voxels.append(addlist)

# Create a conditional list for each voxel defining its own symmetry
max_voxels_ownsym = []
for i in range(len(voxel_count)):
    max_voxels_ownsym.append([element*0 for element in similar_byfull2[i]])

for i in range(len(voxel_count)):
    for j in range(symmetry_limit):
        if voxel_count[i] in similar_byfull2[i][j]:
            max_voxels_ownsym[i][j] = 1
            #if full_symmetry_database[i][j] == '-':
             #   max_voxels_ownsym[i][j] = 0
        else:
            max_voxels_ownsym[i][j] = 0

    setattr(voxels[i],'sym_self', [max_voxels_ownsym[i]])

# Add double symmetries to object handles

In [41]:
# Add double transformation self symmetry to voxel object sym_self handle
for i in range(len(voxels)):
    arr = []
    for j in range(0,12):
        for k in range(j,12):            
            #print (double_symmetry_list[i])
            if (double_symmetry_list[i][j][k] == surroundings[i]).all():
                arr.append([j,k])
    temp = getattr(voxels[i],'sym_self')
    temp.append(arr)
    setattr(voxels[i],'sym_self',temp)

In [42]:
# Function for recording double transformation symmetries to other voxels
def double_trans(input_voxel):
    z = input_voxel.number
    arr = []
    for i in range(len(voxels)):       
        for j in range(0,12):
            for k in range(j,12):
                if i != z and (double_symmetry_list[z][j][k] == surroundings[i]).all():
                    input_voxel.sym_voxels.append([i,[j,k]])
    return

In [43]:
# Run double_trans function
for i in range(len(voxels)):
    double_trans(voxels[i])

# Define voxels attributes for symmetry voxel set and dictionary

In [44]:
# Fill voxel attributes for set of voxels it shares symmetry with and create dictionary for each of these voxels
# and the 'first' symmetry attribute that is shared

for i in voxels:
    for j in i.sym_voxels:
        if j[0] not in i.sym_set:
            i.sym_set.add(j[0])
            i.sym_dict[j[0]] = j[1]

In [45]:
for i in mesovoxel:
    print('number is', i.number)

number is 0
number is 1
number is 2
number is 3
number is 4
number is 7
number is 8
number is 9
number is 10
number is 13
number is 14
number is 15
number is 16
number is 19
number is 20
number is 21
number is 22
number is 25
number is 26
number is 27
number is 28
number is 31
number is 32
number is 33
number is 34
number is 37
number is 38
number is 39
number is 40
number is 43
number is 44
number is 45
number is 46
number is 49
number is 50
number is 51
number is 52
number is 55
number is 56
number is 57
number is 58
number is 61
number is 62
number is 63
number is 64
number is 67
number is 68
number is 69
number is 70


In [46]:
# Remove voxels from minimum central cell based on double transformations
removelist = []
for i in mesovoxel:
    if i in removelist:
        continue
    for j in mesovoxel:     
        if i.number != j.number and j.number in i.sym_set:
            removelist.append(j)
            setattr(j,'uniqueness',[3,i.number,j.sym_dict[i.number]])
    i.uniqueness = [1,i.number,0] # Define object uniqueness for voxels left in mininum central cell

for i in removelist:
    mesovoxel.remove(i)



In [47]:
voxels[4].sym_set

{1,
 7,
 10,
 13,
 16,
 19,
 22,
 25,
 28,
 31,
 34,
 37,
 40,
 43,
 46,
 49,
 52,
 55,
 58,
 61,
 64,
 67,
 70}

# After double (complex) rotations used to remove voxels

In [48]:
unique_voxels_index = []
for i in mesovoxel:
    unique_voxels_index.append(i.number)
print('The list of unique voxels after translation comparisons is', unique_voxels_index)

The list of unique voxels after translation comparisons is [0, 1, 2]


In [49]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

# Paint function for painting bonds between neighboring voxels

In [50]:
# list for calling voxel bonding traits
bond_partners = ["bp_pos_x", "bp_neg_x", "bp_pos_y", "bp_neg_y", "bp_pos_z", "bp_neg_z"]
# Cycle through voxels to check from bonds that do not have color due to complementarity issues
bond_traits = ["bc_pos_x", "bc_neg_x", "bc_pos_y", "bc_neg_y", "bc_pos_z", "bc_neg_z"]
comp_voxels_index = []
NV_comp_voxels_index = []
opp_dir = {0:1, 1:0, 2:3, 3:2, 4:5, 5:4}

def intra_paint(inc_vox, central_cell, bond):
    # the bond input here is either the bond direction number
    bk = 0    
    colors_on_voxel = [0,0,0,0,0,0]
    opp_dir = {0:1, 1:0, 2:3, 3:2, 4:5, 5:4}
    
    # Collect empty bonds in order
    
    #[inc_vox.bc_pos_x, inc_vox.bc_neg_x, inc_vox.bc_pos_y, inc_vox.bc_neg_y, inc_vox.bc_pos_z, inc_vox.bc_neg_z]
    color_count = 0

    i = bond
    for x in range(6):
        colors_on_voxel[x] = getattr(inc_vox,bond_traits[x])[0]

    partner = getattr(inc_vox, bond_partners[i])
    opp_partner = getattr(inc_vox,bond_partners[opp_dir[i]])
    skip = 0
    if voxels[partner] in central_cell:
        if voxels[partner] == voxels[opp_partner] and getattr(inc_vox,bond_traits[opp_dir[i]])[0] != 0:
            skip = 1
            #setattr(inc_vox,bond_traits[i],getattr(inc_vox,bond_traits[opp_dir[i]]))
        
        if getattr(inc_vox, bond_traits[i])[0] == 0 and skip == 0:
            if getattr(voxels[partner], bond_traits[opp_dir[i]])[0] == 0:    
                # for binding complementarity, following loop will change if needed
                comp = 1
                oppcomp = 2

                checklist = [getattr(inc_vox,"bc_pos_x")[1], getattr(inc_vox,"bc_neg_x")[1], getattr(inc_vox,"bc_pos_y")[1],
                             getattr(inc_vox,"bc_neg_y")[1], getattr(inc_vox,"bc_pos_z")[1], getattr(inc_vox,"bc_neg_z")[1]]

                for x in range(6): # loop through binding direction properties contained in 'checklist'
                    if checklist[x] == 2: # if binding of complementarity 2 found on voxel, then all other binding on voxel must be 2
                        comp = 2
                        oppcomp = 1
                        break # once this complementary is found, can break out of for loop

                setattr(inc_vox, bond_traits[i],[color+color_count,comp] ) # set bond attributes on original minimum cell voxel
                setattr(voxels[partner], bond_traits[opp_dir[i]],[color+color_count,oppcomp]) # set bond attributes on original minimum cell voxel
                color_count += 1

            else:                     
                oppcomp = getattr(voxels[partner], bond_traits[opp_dir[i]])[1]
                colorneighb = getattr(voxels[partner], bond_traits[opp_dir[i]])[0]

                if oppcomp == 1:
                    comp = 2
                else:
                    comp = 1
                setattr(inc_vox, bond_traits[i],[colorneighb,comp])
                
                
        
    return color_count # end complementarity function

color = 1 # define starting color for coloring bonds
bindingcolor = {(1,0,0):"bc_pos_x", (-1,0,0):"bc_neg_x", (0,1,0):"bc_pos_y", (0,-1,0):"bc_neg_y",(0,0,1):"bc_pos_z",
                (0,0,-1):"bc_neg_z"} # define dictionary that takes binding directinos and converts it into object properties


# Call paint function
for i in mesovoxel:
    # The i_neighbors list and 'continue' in for loop makes sure edge voxels are not creating two bonds to the same neighbor voxel
    i_neighbors = [0,0,0,0,0,0]
    for loop in range(6):
        a = getattr(i,bond_partners[loop])       
        if a in i_neighbors:
            i_neighbors[loop] = a
            continue
        else:
            i_neighbors[loop] = a
        color_count = 0
        color_count = intra_paint(i, mesovoxel, loop)
        color += color_count     

# Painted intra-mesovoxel colors

In [51]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

# Paint voxel bonds based on voxel symmetries

In [52]:
# Create transformation matrices

trans_rotate_initial = ['bc_pos_x', 'bc_neg_x', 'bc_pos_y', 'bc_neg_y', 'bc_pos_z', 'bc_neg_z']
trans_rotation_x_90 =  [0,0,2,2,-1,-3]
trans_rotation_x_180 = [0,0,1,-1,1,-1]
trans_rotation_x_270 = [0,0,3,1,-2,-2]
trans_rotation_y_90 =  [5,3,0,0,-4,-4]
trans_rotation_y_180 = [1,-1,0,0,1,-1]
trans_rotation_y_270 = [4,4,0,0,-3,-5]
trans_rotation_z_90 =  [2,2,-1,-3,0,0]
trans_rotation_z_180 = [1,-1,1,-1,0,0]
trans_rotation_z_270 = [3,1,-2,-2,0,0]
trans_reflection_x = [1,-1,0,0,0,0]
trans_reflection_y = [0,0,1,-1,0,0]
trans_reflection_z = [0,0,0,0,1,-1]
trans_rotations = [trans_rotation_x_90, trans_rotation_x_180, trans_rotation_x_270, trans_rotation_y_90, trans_rotation_y_180, trans_rotation_y_270, trans_rotation_z_90, trans_rotation_z_180, trans_rotation_z_270, trans_reflection_x, trans_reflection_y, trans_reflection_z]        

def paint_symmetry(inp_vox):
    i = inp_vox
     
    for z in range(12):
        if i.sym_self[0][z] == 1: # Check all self symmetries for given voxel, is a conditional array
            for j in range(6): # Check all bond directions                     
                if getattr(i, bond_traits[j+trans_rotations[z][j]])[0] == 0:
                    setcolor = getattr(i, trans_rotate_initial[j])
                    setattr(i, bond_traits[j+trans_rotations[z][j]], setcolor)
                                       
# Call paint_symmetry function
for i in mesovoxel:
    paint_symmetry(i)

# Voxels colored after painting themselves by their self-symmetry

In [53]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

In [54]:
voxels[0].bc_pos_z

[1, 1]

# Define functions for applying symmetry elements to complementary voxel sets

In [55]:
def apply_transformation(symmetries, check):
      
    vox_1 = symmetries[0]
    vox_2 = symmetries[1]
    symmetry = symmetries[2]
    color_count = 0
    
    if symmetry == 1000:
        
        trans = trans_rotate_initial
           
    elif symmetry != 1000:
      
        rot1 = [0,0,0,0,0,0]

        if type(symmetry) is int:
            first = symmetry
        elif type(symmetry) is list:
            first = symmetry[0]
            second = symmetry[1]
    
        for j in range(6):
            rot1[j+trans_rotations[first][j]] = trans_rotate_initial[j]               
        if type(symmetry) is int:
            trans = rot1
        elif type(symmetry) is list:
            rot2 = [0,0,0,0,0,0]
            for j in range(6):
                rot2[j+trans_rotations[second][j]] = rot1[j]
            trans = rot2 
    
    if check == 0:   
        for j in range(6):

            if getattr(vox_1, trans[j])[0] == 0 and getattr(vox_2, bond_traits[j])[0] != 0: # if vox_1el bond is undefined but original voxel's is defined, take the attributes
                setattr(vox_1, trans[j], getattr(vox_2,bond_traits[j]))
                paint_symmetry(vox_1) # paint new voxel bonds with this new color

            elif getattr(vox_1, trans[j])[0] != 0 and getattr(vox_2, bond_traits[j])[0] == 0:
                setattr(vox_2, bond_traits[j], getattr(vox_1,trans[j]))   
                paint_symmetry(vox_2) # paint original voxel bonds with this new color  
                
                
    elif type(check) is list: # This is for BF = 2
        if symmetries[3] == 0:
            for j in range(6):
                if j in check[0]:
                    comp_bind_dict = {1:2,2:1}
                    temp = getattr(vox_2,bond_traits[j])
                    temp_1 = temp[:]
                    temp_1[1] = comp_bind_dict[temp[1]]
                    setattr(vox_1, trans[j], temp_1)
        else:
            print('HELLO FOR TESTING')
            for j in range(6):
                if j in check[0]:
                    comp_bind_dict = {1:2,2:1}                   
                    temp = getattr(vox_1,trans[j])
                    temp_1 = temp[:]
                    temp_1[1] = comp_bind_dict[temp[1]]
                    setattr(vox_2,bond_traits[j], temp_1)
        
    elif check == 2:
        for j in range(6):
            setattr(vox_1, trans[j], getattr(vox_2,bond_traits[j]))

    return color_count

# Find symmetry voxel for a 'complementary' mesovoxel

In [56]:
def which_symmetry(new_vox): # inputs of two voxels to compare
    sym_elements = []    
    for i in new_vox.sym_set:
        if voxels[i] in mesovoxel:
            sym_elements.append([i,new_vox.sym_dict[i]])
    temp = sym_elements
    if len(temp) > 1:
        if voxels[temp[0][0]].uniqueness[0] == 2:
            sym_elements[0] = temp[1]
            sym_elements[1] = temp[0]
    if new_vox.number == 6:      
        print('sym_elements=', sym_elements)  
    
    return sym_elements

In [57]:
for i in mesovoxel:
    print(i.uniqueness)

[1, 0, 0]
[1, 1, 0]
[1, 2, 0]


In [58]:
voxels[2].sym_set

{3,
 8,
 9,
 14,
 15,
 20,
 21,
 26,
 27,
 32,
 33,
 38,
 39,
 44,
 45,
 50,
 51,
 56,
 57,
 62,
 63,
 68,
 69}

# Mesovoxel Painting Function

In [59]:
check = 0
def mesovox_paint(current_vox):
    global color
    
    # create new voxel for complementarity
    for loop in range(6):             
        # initialize components
        voxel_list = [0, 0, 0, 0] # initialize list to be called later for voxels that are needed for painting colors, but not unique
        
        # PAINTING PROGRAM IS WITHIN THIS IF STATEMENT
        if getattr(current_vox, bond_traits[loop])[0] == 0: # if bond in this direction is not assigned            
            new_vox = voxels[getattr(current_vox,bond_partners[loop])] # set new _vox
            bond_connect = bond_traits[loop] # record binding diretion if needed to call function for wrapped voxel
             
                
            # Create list of which voxels in minimal cell the new voxel has symmetry with    
            symmetry = which_symmetry(new_vox)    
            
            neighb = 0
            if getattr(voxels[symmetry[0][0]],bond_partners[loop]) == new_vox.number: # if the sym_vox is the same as the i voxel in the loop
                neighb = 1 # assigned neighbor as 1
            
            comp_on_comp = 0
            
            if comp_voxels_list != [] and neighb == 1:           
                for j in comp_voxels_list:
                    if symmetry[0][0] == j[1].number and current_vox.number in j[0].neighbors:
                        selected_symmetry = symmetry[1]
                        comp_on_comp = 1
                        
                        #NEW CODING
                        idir = opp_dir[loop]
                        for l in range(6):                         
                            compvoxdir = getattr(j[0],bond_partners[l])                
                            if compvoxdir == current_vox.number:
                                compbinding = l
                        
                        setattr(new_vox, bond_traits[idir],getattr(j[0],bond_traits[compbinding]))
                        
                         # Took this out for testing 1/22/2023
#                         if idir == opp_dir[compbinding]:
#                             setattr(new_vox, bond_traits[idir], getattr(j[0],bond_traits[compbinding]))
                                              
                        break
                    else:
                        selected_symmetry = symmetry[0]
            else:
                selected_symmetry = symmetry[0]

            
            voxel_list = [new_vox, voxels[selected_symmetry[0]], selected_symmetry[1], symmetry]
            
            
            # comp_vox_cell holds the comp_vox_list for all voxels that needed to be added for complementarity reasons and
            # need to be included in the miniumum unit cell
            # NV_comp_vox_cell cells holds the NV_comp_vox_list for all voxels that needed be considered for complementiary reasons
            # but should not be included in the minimum unit cell
            if new_vox.uniqueness[0] != 1 and new_vox.uniqueness[0] != 2:
                if neighb == 1 and comp_on_comp == 0:
                    comp_voxels_index.append(new_vox)
                    comp_voxels_list.append(voxel_list)
                    mesovoxel.append(new_vox)  
                    expanded_central_cell.append(new_vox)
                    new_vox.uniqueness = [2,voxel_list[1].number, voxel_list[2]]
                elif neighb == 0 or comp_on_comp == 1: #new_vox.number in min_voxels_index:
                    NV_comp_voxels_index.append(new_vox)
                    NV_comp_voxels_list.append(voxel_list)
                    expanded_central_cell.append(new_vox)
                    if neighb == 0:
                        new_vox.uniqueness = [3,voxel_list[1].number, voxel_list[2]]
                    elif comp_on_comp == 1:
                        new_vox.uniqueness = [4,voxel_list[1].number, voxel_list[2]]
            
         
            color_count = 0
            apply_transformation(voxel_list, check) # apply rotation and painting functions
            color += color_count    
                
            color_count = 0
            #print('test2 = ', expanded_central_cell)
            color_count = intra_paint(current_vox, expanded_central_cell,loop)
            color += color_count
            
            paint_symmetry(current_vox) # call function to paint original voxel with new bond based on symmetry considerations
            paint_symmetry(new_vox) # call function to paint new voxel with new bond based on symmetry considerations
            
            color_count = 0
            apply_transformation(voxel_list, check) # apply rotation and painting functions
            color += color_count  
                
                
                
              
    return

# Add voxels and binding attributes based on complementarity requirements

In [60]:
# Initialize variables use for the mesovoxel painting


comp_voxels_index = []
comp_voxels_list = []
NV_comp_voxels_index = []
NV_comp_voxels_list = []


similarity_library = []
symmetry_library = []
final_check = 0

expanded_central_cell = mesovoxel[:]

In [61]:
num = 0
stopping = len(mesovoxel)
while num < stopping:   #(min_voxels_index)): # for the voxels currently in mesovoxel
# For program testing
#     if num == 8: # for testing
#         break

    # define current voxel
    c_vox = mesovoxel[num]

    # call mesovoxel_painting program
    mesovox_paint(c_vox)
    # update stopping length for while loop if there is a voxel added to the mininum central cell
    stopping = len(mesovoxel)
    # update while loop counter
    num += 1

In [62]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

# Check to ensure all needed voxels are included in mesovoxel based on final coloring scheme

In [63]:
full_list_voxels = []
mesovoxel_nums = set()
for i in mesovoxel:
    mesovoxel_nums.add(i.number)
    full_list_voxels.append(i)
for i in NV_comp_voxels_index:
    full_list_voxels.append(i)

In [64]:
# update binding list
refresh_binding()

# Function for painting every voxel in system

In [65]:
# Paint all of the voxels based on their mapping symmetries
def paint_system(diffneighb):
    for i in voxels:
        
        if i.uniqueness[0]>2:
            for loop in range(6):
                setattr(i,bond_traits[loop],[0,0])
    
    comp_bind_dict = {1:2,2:1}

    check = 2
    paint_all = voxels[:] # Hard copy all voxels to another list
    painted = []
    
    what = []
#     for i in paint_all:
#         what.append(i.number)
#     print('what = ',what)
    a = 0
    b = 0
    c = 0
    
    stop = 0
    while len(painted)<len(paint_all):
        if stop ==1:
            break
        for i in paint_all:
#             if voxels[6] in painted:
#                 stop = 1
#                 break
            if i in painted:
                continue
            #print('voxel is',i.number)
#             if i == voxels[3]:
#                 print('awesome')
#                 print('uniqueness',i.uniqueness)
#                 print('coorda',i.coordinates[0])
#             print('abc',[a,b,c])
#             if i in mesovoxel:
#                 paint_all.remove(i)
#                 painted.append(i)
#                 continue
            if i.coordinates[0] == a or i.coordinates[1] == b or i.coordinates[2] == c:
                voxel_list = [0,0,0,0]
#                 if i == paint_all:
#                     print('voxelno = ',i)
                if i.uniqueness[0] == 1 or i.uniqueness[0] == 2:
                    painted.append(i)
#                     if i == voxels[3]:
#                         print('voxelyes = ',i)
                    continue    
        #         elif i.uniqueness[0] == 3 or i.uniqueness[0] == 4:
        #             selected_symmetry = [i.uniqueness[1],i.uniqueness[2]]
        #             voxel_list = [i, voxels[selected_symmetry[0]], selected_symmetry[1], 0]
        #             apply_transformation(voxel_list, check) # apply rotation and painting functions

                else:   
                    
                    symmetry = which_symmetry(i)    
                    selected_symmetry = symmetry[0] 
                    voxel_list = [i, voxels[selected_symmetry[0]], selected_symmetry[1], 0]
                    i.uniqueness = [5,voxels[selected_symmetry[0]].number,selected_symmetry[1]]
                    apply_transformation(voxel_list, check) # apply rotation and painting functions
                    
                   
                    if i == voxels[6]:
                        print('voxel = ',i.number)
                        print('selected_symmetry =',selected_symmetry)
#                         painted.append(i)
#                         continue
                    for loop in range(6):
                        if diffneighb == 1:
                            if i.coordinates[0] == 0 and loop == 1:
                                continue
                            elif i.coordinates[0] == (rows - 1) and loop == 0:
                                continue
                            elif i.coordinates[1] == 0 and loop == 3:
                                continue                              
                            elif i.coordinates[1] == (columns - 1) and loop == 2:
                                continue
                            elif i.coordinates[2] == 0 and loop == 5:
                                continue
                            elif i.coordinates[2] == (layers - 1) and loop == 4:
                                continue    
            
                        bpartner = voxels[getattr(i,bond_partners[loop])]
                        bond1comp = getattr(i,bond_traits[loop])
                        if getattr(bpartner,bond_traits[opp_dir[loop]])[0] == 0:
                            continue
                        else:
                            bond2comp = [getattr(bpartner,bond_traits[opp_dir[loop]])[0], comp_bind_dict[getattr(bpartner,bond_traits[opp_dir[loop]])[1]]]

                        if i == voxels[6]:
                            print('bond1comp = ',bond1comp)
                            print('bond2comp =',bond2comp)
                            print('partner =',bpartner.number)
                        if bond1comp != bond2comp:    
                            selected_symmetry = symmetry[1]
                            voxel_list = [i, voxels[selected_symmetry[0]], selected_symmetry[1], 0]
                            i.uniqueness = [5,voxels[selected_symmetry[0]].number,selected_symmetry[1]]
                            apply_transformation(voxel_list, check) # apply rotation and painting functions
                            break
                            
                painted.append(i)
#                 if i.number == 34:
#                     print('yes')
                
        if a < rows:
            a += 1
        if b < columns:
            b += 1
        if c < layers:
            c += 1

        #print('length paint_all', paint_all[1].number)
        #print('a =',[a,b,c])
        print('painted =',len(painted))
        #print('paint_all =',len(paint_all))

    return

In [66]:
# run paint system function
paint_system(0)

sym_elements= [[0, 1000], [24, 1000]]
voxel =  6
selected_symmetry = [0, 1000]
bond1comp =  [3, 1]
bond2comp = [3, 2]
partner = 0
painted = 42
painted = 64
painted = 72


In [67]:
voxels[0].bp_pos_y

6

In [68]:
showcolors(voxels)

<IPython.core.display.Javascript object>

# Functions for BF2 and BF3

In [69]:
def new_map(inc_voxel,list_for_check):

    for k in mesovoxel:
        #print('new_map_beginning')
        if k.uniqueness[0] == 2 and k.uniqueness[1] == inc_voxel.number:
            #print('listforcheck =', list_for_check)
            selected_symmetry = [k.uniqueness[1],k.uniqueness[2]]
            voxel_list = [k, voxels[selected_symmetry[0]], selected_symmetry[1], 0]
            apply_transformation(voxel_list, list_for_check)
            
#         if inc_voxel.uniqueness > 1 and inc_voxel.uniqueness[1] == k.number
#             print('new_map_secondpart')
#             selected_symmetry = [inc_voxel.uniqueness[1],inc_voxel.uniqueness[2]]
#             voxel_list = [inc_voxel, voxels[selected_symmetry[0]], selected_symmetry[1], -1]
#             apply_transformation(voxel_list, list_for_check)
    return


def write_over(inc_vox,new_color,old_color,check):
    
    checklist = [getattr(inc_vox,"bc_pos_x"), getattr(inc_vox,"bc_neg_x"), getattr(inc_vox,"bc_pos_y"),
                getattr(inc_vox,"bc_neg_y"), getattr(inc_vox,"bc_pos_z"), getattr(inc_vox,"bc_neg_z")]

    for i in range(6):
        if check == -1:        
            if checklist[i][0] == old_color:
                checklist[i] = [new_color,checklist[i][1]]
                inc_vox.bond_change_dict[new_color] = old_color
        
        if check != -1:
            if checklist[i][0] == old_color and voxels[getattr(inc_vox,bond_partners[i])] == check:
                checklist[i] = [new_color,checklist[i][1]]
    
    setattr(inc_vox,"bc_pos_x",checklist[0])
    setattr(inc_vox,"bc_neg_x",checklist[1])
    setattr(inc_vox,"bc_pos_y",checklist[2])
    setattr(inc_vox,"bc_neg_y",checklist[3])
    setattr(inc_vox,"bc_pos_z",checklist[4])
    setattr(inc_vox,"bc_neg_z",checklist[5])
    
    return

# Function for BF2 calculation

In [70]:
def BF2run():   
    paint_all = voxels[:]
    global color
    
    x_rot = ['bc_pos_y','bc_pos_z','bc_neg_y','bc_neg_z']
    y_rot = ['bc_pos_z','bc_pos_x','bc_neg_z','bc_neg_x']
    z_rot = ['bc_pos_x','bc_pos_y','bc_neg_x','bc_neg_y']
    x_rot_check = ['bc_pos_y','bc_pos_z','bc_neg_y','bc_neg_z','bc_pos_y','bc_pos_z','bc_neg_y','bc_neg_z']
    y_rot_check = ['bc_pos_z','bc_pos_x','bc_neg_z','bc_neg_x','bc_pos_z','bc_pos_x','bc_neg_z','bc_neg_x']
    z_rot_check = ['bc_pos_x','bc_pos_y','bc_neg_x','bc_neg_y','bc_pos_x','bc_pos_y','bc_neg_x','bc_neg_y']


    if BF == 2 or BF == 3:

        for i in mesovoxel:
            temp = [0,0,0,0,0,0,0,0]
            if i.uniqueness[0] == 1:
                for j in range(color+1):
                    colorlist = [0,0,0,0,0,0]
                    colorcount = [0,0,0,0,0,0]
                    counterlist = [-1,-1,-1,-1,-1,-1]
                    for k in range(6):
                        if i.binding_list[k][0] == j:
                            colorlist[k] = 1

                    #print('colorlist =',colorlist)

                    for k in range(6):
                        for l in range(k,6):
                            #print('k = ',k)
                            #print('l = ',l)
                            if l == k:
                                continue
                            elif colorlist[k] == 1 and colorlist[l] == 1:                  
                                #print('yes')
                                if k == 0 or k == 1:
                                    uselistk = x_rot_check                      
                                elif k == 2 or k == 3:
                                    uselistk = y_rot_check
                                elif k == 4 or k == 5:
                                    uselistk = z_rot_check


                                if l == 0 or l == 1:
                                    uselistl = x_rot                      
                                elif l == 2 or l == 3:
                                    uselistl = y_rot
                                elif l == 4 or l == 5:
                                    uselistl = z_rot

                                breakloop = 0
                                for start in range(4):
                                    if breakloop == 1:
                                        break
                                    #print('hello')
                                    checklist = uselistk[start:(start+4)]

                                    list1 = [0,0,0,0]
                                    list2 = [0,0,0,0]                     
                                    for listfill in range(4):
                                        list1[listfill] = getattr(i,checklist[listfill])
                                        list2[listfill] = getattr(i,uselistl[listfill])

        #                             if k == 0 and l == 4:
        #                                 print('list 1 = ', list1)
        #                                 print('list 2 =', list2)
                                    if list1 == list2:
    #                                     if k == 4 and l == 5:
    #                                         print('list 1 = ', list1)
    #                                         print('list 2 =', list2)
                                        if colorcount[k] == 0:
                                            colorcount[k] += 1
                                        colorcount[k] += 1
                                        colorlist[l] = 0
                                        counterlist[l] = k                                    
                                       
                                        breakloop = 1
                                        break
                            elif l == 5 and colorlist[k] != 0 and colorcount[k] == 0:
                                colorcount[k] += 1


                   
                    # go to back the colors for loop

                    maxcolor = max(colorcount)
                    changelist = []          
                    for change in reversed(range(6)):

                        #print ('change =', change)
                        # Change color
                        if colorcount[change] != maxcolor and colorcount[change] != 0:
                            #print('hello')
                            temp = getattr(i,bond_traits[change])
                            temp[0] = color
                            setattr(i,bond_traits[change],temp)
                            changelist.append(change)

                            for j in range(6):
                                if counterlist[j] == change:
                                    setattr(i,bond_traits[j][0],color)
                                    changelist.append(j)

                            checklist = [changelist, color]
                            new_map(i,checklist)
                            color += 1
        for i in voxels:
            temp = []
            for j in range(6):
                temp.append(getattr(i,bond_traits[j]))

            i.binding_list = temp



#     for i in paint_all:
#         print(i.uniqueness[0])
#         print('voxel = ', i.number)
#         if i.uniqueness[0] > 2:
#             selected_symmetry = [i.uniqueness[1],i.uniqueness[2]]
#             voxel_list = [i, voxels[selected_symmetry[0]], selected_symmetry[1], 0]
#             apply_transformation(voxel_list, check) # apply rotation and painting functions    
    for i in voxels:
        
        if i.uniqueness[0]>2:
            for loop in range(6):
                setattr(i,bond_traits[loop],[0,0])
    #paint_system()
    

    # update binding list
    refresh_binding()
    
    return

In [71]:
voxels[0].uniqueness

[1, 0, 0]

# Function for BF3 calculation

In [72]:
def BF3run():
    global color
    comp_bind_dict = {1:2,2:1}
    if BF == 3:
        ignorelist = []
        for i in mesovoxel:
            if i.uniqueness[0] == 2:
                bondrecord = []
                colorchangeset = []
                colorcount = []
                partnerchangeset = []
                partneruniqueness = []
                partnersym = []
                parentcompare = voxels[i.uniqueness[1]]
                for j in range(6):
                    partner = voxels[getattr(i,bond_partners[j])]
                    bondchange = getattr(i,bond_traits[j])
                    oppbondchange = [bondchange[0],comp_bind_dict[bondchange[1]]]
                    bondcompare = 0
                    #parentcompare = voxels[i.uniqueness[1]]
                    for l in range(6):
                        if oppbondchange == getattr(parentcompare,bond_traits[l]):
                            bondcompare = 1
                            break
                    if bondcompare == 1:
                        continue
                    #partner uniqueness to i.number is meaninglues right below
                    if partner.uniqueness[1] != i.uniqueness[1] and partner.uniqueness != i.number and bondchange[0] not in i.bond_change_dict:
                        bondchange = getattr(i,bond_traits[j])
                        colorcount.append(bondchange[0])
                        #uniquemeas = getattr(partner,"uniqueness")[1]
                        if bondchange[0] not in colorchangeset:
                            colorchangeset.append(bondchange[0])
                            bondrecord.append(bondchange)
                        if getattr(i,bond_partners[j]) not in partnerchangeset:# and uniquemeas not in partnersym:    
                            partnerchangeset.append(partner.number)
                            partneruniqueness.append(partner.uniqueness[0])
                            #partnersym.append(uniquemeas)
                
                
                count = 0
                for j in colorchangeset:
                    if colorcount.count(j) < BFlimit:
                        count += 1
                        continue
                    write_over(i,color,j,-1)
                    for l in range(len(colorcount)):
                        if colorcount[l] == j:
                            write_over(voxels[partnerchangeset[l]],color,j,-1)
                        if voxels[partnerchangeset[l]].uniqueness[0]>2:
                            fixmeso = voxels[voxels[k].uniqueness[1]]
                            voxel_list = [fixmeso,voxels[k],fixmeso.sym_dict[k],0]
                            apply_transformation(voxel_list,2)
                            refresh_binding()
                    color += 1
    # Run neighbors function
    neighbors(1)   # Sends with neighbor 1   
    
    #erase and paint all binding characteristics for non-mesovoxel voxels
    for i in voxels:
        if i.uniqueness[0]>2:
            for loop in range(6):
                setattr(i,bond_traits[loop],[0,0])
    #paint_system()
    
    # update binding list
    refresh_binding()
    
    return

# Function for calculating origami ratios

In [73]:
def conc_ratios():

    list_ratio = []
    conc_list = [0]*len(mesovoxel)
#     test_list = [0]*len(mesovoxel)
    for i in reduced_central_cell:
        if i in mesovoxel:
            list_ratio.append(i.number)
        else:
            list_ratio.append(i.uniqueness[1])

    for i in range(len(mesovoxel)):
        conc_list[i] = list_ratio.count(mesovoxel[i].number)
        if conc_list[i] == 0:
            conc_list[i] = 1
#             test_list[i] = 1
            

    lowest = min(conc_list)
    final_concs = [element/lowest for element in conc_list]    


    for i in range(len(mesovoxel)):
        print('Origami {} requires a {}x concentration'.format(mesovoxel[i].number, final_concs[i]))
        
    return final_concs

# Function for saving outputs

In [74]:
def outputsave(voxel_objects, conc_list=None,full_list=False ):
    temp = [i.number for i in voxel_objects]
    BFxorigami = copy.deepcopy(temp)

    temp = [i.coordinates for i in voxel_objects]
    BFxcoordinates = copy.deepcopy(temp)

    temp = [[f'{j[0]}' if j[1] == 1 else f'{j[0]}' +'C' for j in i.binding_list] for i in voxel_objects]
    
    BFxbinding = copy.deepcopy(temp)
    
    temp = [f'{int(i.material)}' for i in voxel_objects]
    BFxmaterial = copy.deepcopy(temp)
    
    if full_list == True:
        temp = [i.uniqueness[1] for i in voxel_objects]
        BFxoriginal = temp
        outputdict = {'Origami':BFxorigami,'Coordinates':BFxcoordinates,'Original':BFxoriginal,\
                      'bc_pos_x':[f'{k[0] if len(k[0])==3 else "0"+k[0]}' for k in BFxbinding],\
                      'bc_neg_x':[f'{k[1] if len(k[1])==3 else "0"+k[1]}' for k in BFxbinding],\
                      'bc_pos_y':[f'{k[2] if len(k[2])==3 else "0"+k[2]}' for k in BFxbinding],\
                      'bc_neg_y':[f'{k[3] if len(k[3])==3 else "0"+k[3]}' for k in BFxbinding],\
                      'bc_pos_z':[f'{k[4] if len(k[4])==3 else "0"+k[0]}' for k in BFxbinding],
                      'bc_neg_z':[k[5] for k in BFxbinding]}
    else:
        temp = [i for i in conc_list]
        BFxratios = copy.deepcopy(temp)

        outputdict = {'Origami':BFxorigami,'Coordinates':BFxcoordinates,'Concentration':BFxratios, 'Material':BFxmaterial,\
                      'bc_pos_x':[k[0] for k in BFxbinding],'bc_neg_x':[k[1] for k in BFxbinding],\
                      'bc_pos_y':[k[2] for k in BFxbinding],'bc_neg_y':[k[3] for k in BFxbinding],\
                      'bc_pos_z':[k[4] for k in BFxbinding],'bc_neg_z':[k[5] for k in BFxbinding]}
    df=pd.DataFrame(outputdict)
    
    return df


#  MINIMAL SET OF VOXELS AND BINDING COLORS/COMPLEMENTS - Binding Flexibility 1

In [75]:
# # ALL THE VOXELS AND BONDS THAT WERE NEEDED TO PAINT THE MESOVOXEL
# showcolors(full_list_voxels)

In [76]:
# # Visualize voxels and color interactions in expanded system (the mesovoxel after translational symmetry reduction)
# showcolors(reduced_central_cell)

# Mesovoxel Binding Flexibility 1

In [77]:
voxels[3].neighbors

[27, 27, 9, 15, 4, 2]

In [78]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

In [79]:
print('THE NUMBER OF UNIQUE ORIGAMI USED IN THIS DESIGN IS', len(mesovoxel), 'ORIGAMI')
print('THE NUMBER OF BINDING COLORS USED IN THIS DESIGN IS', color-1, 'COLORS')

THE NUMBER OF UNIQUE ORIGAMI USED IN THIS DESIGN IS 6 ORIGAMI
THE NUMBER OF BINDING COLORS USED IN THIS DESIGN IS 5 COLORS


# View of entire voxel set with binding colors - Binding Flexibility 1

In [80]:
showcolors(voxels)

<IPython.core.display.Javascript object>

In [81]:
# update binding list
refresh_binding()

In [82]:
showcolors(reduced_central_cell)

NameError: name 'reduced_central_cell' is not defined

In [ ]:
for i in voxels:
    if i.uniqueness[1] == 12 or i.uniqueness[1] == 13:
        print(i.number,i.uniqueness)

In [ ]:
final_concs=conc_ratios()
print(final_concs)

# Save output from Binding Flexibility 1

In [ ]:
df=outputsave(mesovoxel,final_concs)
df.to_excel(f'{DesignName}_BF1.xlsx',index=False)
df=outputsave(reduced_central_cell,full_list=True)
df

# Analysis of binding for Binding Flexibility 2

In [ ]:
BF=3

In [ ]:
type(voxels[2].uniqueness[0])

In [ ]:
# Run function for binding flexibility 2
BF2run() 

# MINIMAL SET OF VOXELS AND BINDING COLORS/COMPLEMENTS - Binding Flexibility 2

In [83]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

In [84]:
mesovoxel

In [85]:
mesovoxel[2].number

2

In [89]:
voxels[2].bc_pos_x

[5, 1]

In [196]:
print('THE NUMBER OF UNIQUE ORIGAMI USED IN THIS DESIGN IS', len(mesovoxel), 'ORIGAMI')
print('THE NUMBER OF BINDING COLORS USED IN THIS DESIGN IS', color-1, 'COLORS')

THE NUMBER OF UNIQUE ORIGAMI USED IN THIS DESIGN IS 4 ORIGAMI
THE NUMBER OF BINDING COLORS USED IN THIS DESIGN IS 2 COLORS


In [197]:
final_concs=conc_ratios()

Origami 0 requires a 1.0x concentration
Origami 1 requires a 1.0x concentration
Origami 12 requires a 1.0x concentration
Origami 13 requires a 1.0x concentration


# Save output from Binding Flexibility 2

In [198]:
df=outputsave(mesovoxel,final_concs)
df.to_excel(f'{DesignName}_BF2.xlsx',index=False)
df=outputsave(reduced_central_cell,full_list=True)
df

,Origami,Coordinates,Original,bc_pos_x,bc_neg_x,bc_pos_y,bc_neg_y,bc_pos_z,bc_neg_z
0,0,"(0, 0, 0)",0,01,01,01,01,01,1
1,1,"(0, 0, 1)",1,00C,00C,00C,00C,00C,1C
2,12,"(1, 1, 0)",12,00C,00C,00C,00C,00C,2
3,13,"(1, 1, 1)",0,02C,02C,02C,02C,02C,2C


# View of entire voxel set with binding colors - Binding Flexibility 2

In [73]:
paint_system(0)
showcolors(voxels)

painted = 44
painted = 68
painted = 78
painted = 80


<IPython.core.display.Javascript object>

In [74]:
# Run binding flexibility 3 function
BF3run()

In [75]:
paint_system(1)

painted = 44
painted = 68
painted = 78
painted = 80


# MINIMAL SET OF VOXELS AND BINDING COLORS/COMPLEMENTS - Binding Flexibility 3

In [76]:
voxels[8].uniqueness

[2, 7, 4]

In [77]:
showcolors(mesovoxel)

<IPython.core.display.Javascript object>

In [78]:
print('THE NUMBER OF UNIQUE ORIGAMI USED IN THIS DESIGN IS', len(mesovoxel), 'ORIGAMI')
print('THE NUMBER OF BINDING COLORS USED IN THIS DESIGN IS', color-1, 'COLORS')

THE NUMBER OF UNIQUE ORIGAMI USED IN THIS DESIGN IS 9 ORIGAMI
THE NUMBER OF BINDING COLORS USED IN THIS DESIGN IS 17 COLORS


# View of entire voxel set with binding colors - Binding Flexibility 3

In [79]:
showcolors(voxels)

<IPython.core.display.Javascript object>

# Concentrations of Origami for Synthesis

In [80]:
voxels[7].bp_pos_z

8

In [81]:
voxels[8].uniqueness

[2, 7, 4]

In [82]:
final_concs=conc_ratios()

Origami 0 requires a 1.0x concentration
Origami 5 requires a 2.0x concentration
Origami 7 requires a 2.5x concentration
Origami 25 requires a 2.0x concentration
Origami 26 requires a 4.0x concentration
Origami 27 requires a 2.0x concentration
Origami 1 requires a 1.0x concentration
Origami 6 requires a 2.0x concentration
Origami 8 requires a 1.5x concentration


# Save output from Binding Flexibility 3

In [83]:
df=outputsave(mesovoxel,final_concs)
df.to_excel(f'{DesignName}_BF3.xlsx',index=False)

df=outputsave(reduced_central_cell,full_list=True)
df


,Origami,Coordinates,Original,bc_pos_x,bc_neg_x,bc_pos_y,bc_neg_y,bc_pos_z,bc_neg_z
0,0,"(0, 0, 0)",0,01A,06A,01A,06A,01A,7A
1,1,"(0, 0, 1)",0,16A,17A,17A,16A,016A,7B
2,2,"(0, 0, 2)",0,00B,00B,00B,00B,00B,0B
3,3,"(0, 0, 3)",1,00B,00B,00B,00B,00B,0B
4,5,"(0, 1, 0)",5,02B,08B,09B,01B,10B,11B
5,6,"(0, 1, 1)",5,08B,02B,09B,17B,11B,10A
6,7,"(0, 1, 2)",7,03A,12B,09A,06B,13B,11A
7,8,"(0, 1, 3)",7,12B,03A,09A,16B,11A,13A
8,10,"(0, 2, 0)",7,00B,00B,00B,00B,00B,0B
9,11,"(0, 2, 1)",8,00B,00B,00B,00B,00B,0B


In [84]:
which_symmetry(voxels[8])

[[7, 4]]

In [85]:
from itertools import permutations
perm = permutations([1,0],6)
for i in list(perm):
    print(i)

In [259]:
voxels[0].surround_matrix

array([[[0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 1., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0

404 GET / (130.199.254.63) 34.68ms
404 GET / (130.199.254.63) 1.00ms
404 GET / (130.199.254.63) 1.00ms
404 GET /properties/description.dhtml (130.199.254.63) 0.35ms
404 GET /properties/configuration.php?tab=Status (130.199.254.63) 0.72ms
404 GET /header.php?tab=status (130.199.254.63) 0.00ms
404 GET / (130.199.254.63) 1.00ms
404 GET / (130.199.254.63) 0.00ms
404 GET / (130.199.254.63) 1.01ms
404 GET / (130.199.254.63) 0.00ms
404 GET / (130.199.254.63) 0.00ms
404 GET / (130.199.254.63) 0.00ms
404 GET /login.php (130.199.254.63) 0.00ms
404 GET /this_server/all_settings.shtml (130.199.254.63) 1.00ms
404 GET / (130.199.254.63) 0.00ms
404 GET /authenticate/login (130.199.254.63) 0.00ms
404 GET /start.js (130.199.254.63) 1.00ms
404 GET /tmui/ (130.199.254.63) 1.00ms
404 GET /login (130.199.254.63) 0.00ms
404 GET /netmri/config/userAdmin/login.tdf (130.199.254.63) 0.00ms
404 GET /scgi-bin/platform.cgi (130.199.254.63) 0.00ms
404 GET /admin/login.do (130.199.254.63) 0.00ms
404 GET /en/main.js 

404 GET /sslkeys/ (130.199.254.63) 1.00ms
404 GET /stat/ (130.199.254.63) 0.00ms
404 GET /statistic/ (130.199.254.63) 1.00ms
404 GET /statistics/ (130.199.254.63) 0.00ms
404 GET /stats/ (130.199.254.63) 0.00ms
404 GET /stats_old/ (130.199.254.63) 0.00ms
404 GET /sys/ (130.199.254.63) 0.00ms
404 GET /sysadmin/ (130.199.254.63) 0.00ms
404 GET /sysbackup/ (130.199.254.63) 1.00ms
404 GET /test/ (130.199.254.63) 1.00ms
404 GET /testing/ (130.199.254.63) 0.00ms
404 GET /tests/ (130.199.254.63) 0.00ms
404 GET /tmp/ (130.199.254.63) 0.00ms
404 GET /userdb/ (130.199.254.63) 0.00ms
404 GET /users/ (130.199.254.63) 1.00ms
404 GET /ustats/ (130.199.254.63) 0.00ms
404 GET /web_usage/ (130.199.254.63) 0.00ms
404 GET /webaccess/ (130.199.254.63) 0.00ms
404 GET /webadmin/ (130.199.254.63) 0.00ms
404 GET /webalizer/ (130.199.254.63) 1.00ms
404 GET /webstat/ (130.199.254.63) 1.00ms
404 GET /webstats/ (130.199.254.63) 0.00ms
404 GET /webtrends/ (130.199.254.63) 0.00ms
404 GET /wstats/ (130.199.254.63) 0.

404 GET /eforum/ (130.199.254.63) 0.00ms
404 GET /ejemplo/ (130.199.254.63) 0.00ms
404 GET /ejemplos/ (130.199.254.63) 0.00ms
404 GET /emailclass/ (130.199.254.63) 0.00ms
404 GET /employees/ (130.199.254.63) 0.00ms
404 GET /empoyees/ (130.199.254.63) 1.00ms
404 GET /empris/ (130.199.254.63) 0.00ms
404 GET /envia/ (130.199.254.63) 0.00ms
404 GET /enviamail/ (130.199.254.63) 0.00ms
404 GET /error/ (130.199.254.63) 0.00ms
404 GET /errors/ (130.199.254.63) 0.00ms
404 GET /es/ (130.199.254.63) 1.00ms
404 GET /estmt/ (130.199.254.63) 0.00ms
404 GET /etc/ (130.199.254.63) 0.00ms
404 GET /example/ (130.199.254.63) 0.00ms
404 GET /examples/ (130.199.254.63) 0.00ms
404 GET /exc/ (130.199.254.63) 0.00ms
404 GET /excel/ (130.199.254.63) 1.00ms
404 GET /exchange/ (130.199.254.63) 0.00ms
404 GET /exe/ (130.199.254.63) 1.00ms
404 GET /exec/ (130.199.254.63) 0.00ms
404 GET /export/ (130.199.254.63) 0.00ms
404 GET /external/ (130.199.254.63) 0.00ms
404 GET /f/ (130.199.254.63) 0.00ms
404 GET /fbsd/ (13

404 GET /purchases/ (130.199.254.63) 0.00ms
404 GET /pw/ (130.199.254.63) 1.00ms
404 GET /random_banner/ (130.199.254.63) 0.00ms
404 GET /rdp/ (130.199.254.63) 1.00ms
404 GET /register/ (130.199.254.63) 0.00ms
404 GET /registered/ (130.199.254.63) 1.00ms
404 GET /report/ (130.199.254.63) 0.00ms
404 GET /reports/ (130.199.254.63) 0.00ms
404 GET /reseller/ (130.199.254.63) 0.00ms
404 GET /restricted/ (130.199.254.63) 0.00ms
404 GET /retail/ (130.199.254.63) 0.00ms
404 GET /reviews/ (130.199.254.63) 1.00ms
404 GET /root/ (130.199.254.63) 1.00ms
404 GET /rsrc/ (130.199.254.63) 0.00ms
404 GET /sales/ (130.199.254.63) 0.00ms
404 GET /sample/ (130.199.254.63) 0.00ms
404 GET /samples/ (130.199.254.63) 0.00ms
404 GET /save/ (130.199.254.63) 0.00ms
404 GET /script/ (130.199.254.63) 1.00ms
404 GET /scripts/ (130.199.254.63) 0.00ms
404 GET /search/ (130.199.254.63) 0.00ms
404 GET /search-ui/ (130.199.254.63) 1.00ms
404 GET /sell/ (130.199.254.63) 0.00ms
404 GET /server-info/ (130.199.254.63) 0.00m

...

404 GET /hostingcontroller/ (130.199.254.68) 0.00ms
404 GET /hr/ (130.199.254.68) 1.00ms
404 GET /ht/ (130.199.254.68) 0.00ms
404 GET /html/ (130.199.254.68) 0.00ms
404 GET /hyperstat/ (130.199.254.68) 0.00ms
404 GET /ibank/ (130.199.254.68) 0.00ms
404 GET /ibill/ (130.199.254.68) 0.00ms
404 GET /icons/ (130.199.254.68) 0.00ms
404 GET /idea/ (130.199.254.68) 0.00ms
404 GET /ideas/ (130.199.254.68) 1.00ms
404 GET /image/ (130.199.254.68) 0.97ms
404 GET /imagenes/ (130.199.254.68) 0.99ms
404 GET /imagery/ (130.199.254.68) 0.00ms
404 GET /images/ (130.199.254.68) 1.00ms
404 GET /img/ (130.199.254.68) 0.00ms
404 GET /imp/ (130.199.254.68) 0.00ms
404 GET /import/ (130.199.254.68) 0.00ms
404 GET /impreso/ (130.199.254.68) 0.00ms
404 GET /inc/ (130.199.254.68) 0.00ms
404 GET /info/ (130.199.254.68) 0.00ms
404 GET /information/ (130.199.254.68) 0.00ms
404 GET /ingresa/ (130.199.254.68) 1.00ms
404 GET /ingreso/ (130.199.254.68) 1.00ms
404 GET /install/ (130.199.254.68) 0.00ms
404 GET /internal/

404 GET /source/ (130.199.254.68) 0.00ms
404 GET /sql/ (130.199.254.68) 0.00ms
404 GET /squid/ (130.199.254.68) 0.00ms
404 GET /src/ (130.199.254.68) 0.00ms
404 GET /srchadm/ (130.199.254.68) 0.00ms
404 GET /staff/ (130.199.254.68) 0.00ms
404 GET /stats-bin-p/ (130.199.254.68) 0.00ms
404 GET /status/ (130.199.254.68) 0.00ms
404 GET /storage/ (130.199.254.68) 1.00ms
404 GET /store/ (130.199.254.68) 1.00ms
404 GET /storemgr/ (130.199.254.68) 0.00ms
404 GET /stronghold-info/ (130.199.254.68) 0.00ms
404 GET /stronghold-status/ (130.199.254.68) 1.00ms
404 GET /stuff/ (130.199.254.68) 1.00ms
404 GET /style/ (130.199.254.68) 1.00ms
404 GET /styles/ (130.199.254.68) 0.00ms
404 GET /stylesheet/ (130.199.254.68) 0.00ms
404 GET /stylesheets/ (130.199.254.68) 0.00ms
404 GET /subir/ (130.199.254.68) 0.00ms
404 GET /sun/ (130.199.254.68) 0.00ms
404 GET /super_stats/ (130.199.254.68) 0.00ms
404 GET /support/ (130.199.254.68) 0.00ms
404 GET /supporter/ (130.199.254.68) 0.00ms
404 GET /system/ (130.199

404 GET /acal/ (130.199.254.68) 1.00ms
404 GET /actualanalyzer/ (130.199.254.68) 0.00ms
404 GET /admbook/ (130.199.254.68) 0.00ms
404 GET /ads/ (130.199.254.68) 0.00ms
404 GET /adserver/ (130.199.254.68) 0.00ms
404 GET /agenda/ (130.199.254.68) 0.00ms
404 GET /agora/ (130.199.254.68) 0.00ms
404 GET /album/ (130.199.254.68) 0.00ms
404 GET /albums/ (130.199.254.68) 0.00ms
404 GET /amazon/ (130.199.254.68) 0.00ms
404 GET /amember/ (130.199.254.68) 0.00ms
404 GET /amserver/ (130.199.254.68) 0.00ms
404 GET /angeline/ (130.199.254.68) 0.00ms
404 GET /articles/ (130.199.254.68) 0.00ms
404 GET /asteridex/ (130.199.254.68) 0.00ms
404 GET /auction/ (130.199.254.68) 0.00ms
404 GET /auktion/ (130.199.254.68) 0.00ms
404 GET /awstats/ (130.199.254.68) 0.00ms
404 GET /awstats-cgi/ (130.199.254.68) 0.00ms
404 GET /awstats/cgi-bin/ (130.199.254.68) 0.00ms
404 GET /bannerexchange/ (130.199.254.68) 1.00ms
404 GET /base/ (130.199.254.68) 0.00ms
404 GET /basilix/ (130.199.254.68) 0.00ms
404 GET /bitweaver/

404 GET /mnemo/ (130.199.254.68) 0.00ms
404 GET /modules/forum/ (130.199.254.68) 0.00ms
404 GET /modx/ (130.199.254.68) 0.00ms
404 GET /moinmoin/ (130.199.254.68) 0.00ms
404 GET /moodle/ (130.199.254.68) 0.00ms
404 GET /mrbs/ (130.199.254.68) 0.00ms
404 GET /mt/ (130.199.254.68) 0.00ms
404 GET /mvnforum/ (130.199.254.68) 0.00ms
404 GET /mybb/ (130.199.254.68) 0.00ms
404 GET /myreview/ (130.199.254.68) 0.00ms
404 GET /nag/ (130.199.254.68) 0.00ms
404 GET /netoffice/ (130.199.254.68) 0.00ms
404 GET /netofficedwins/ (130.199.254.68) 0.00ms
404 GET /newsfeeds/ (130.199.254.68) 0.00ms
404 GET /noahsclassifieds/ (130.199.254.68) 0.00ms
404 GET /nocc/ (130.199.254.68) 0.00ms
404 GET /nucleus/ (130.199.254.68) 0.00ms
404 GET /nuked-clan/ (130.199.254.68) 0.00ms
404 GET /nukedit/ (130.199.254.68) 1.00ms
404 GET /observer/ (130.199.254.68) 0.00ms
404 GET /ocs/ (130.199.254.68) 1.00ms
404 GET /oempro/ (130.199.254.68) 0.00ms
404 GET /oneorzero/ (130.199.254.68) 1.00ms
404 GET /onlineviewing/ (130

404 GET /tinywebgallery/ (130.199.254.68) 1.00ms
404 GET /toplist/ (130.199.254.68) 0.00ms
404 GET /topsite/ (130.199.254.68) 0.00ms
404 GET /topsites/ (130.199.254.68) 0.00ms
404 GET /trac/ (130.199.254.68) 0.00ms
404 GET /ttforum/ (130.199.254.68) 0.00ms
404 GET /turba/ (130.199.254.68) 0.00ms
404 GET /twg/ (130.199.254.68) 0.00ms
404 GET /twiki/bin/ (130.199.254.68) 1.00ms
404 GET /typo3/ (130.199.254.68) 1.00ms
404 GET /typolight/ (130.199.254.68) 0.00ms
404 GET /ubbthreads/ (130.199.254.68) 1.00ms
404 GET /uebimiau/ (130.199.254.68) 0.00ms
404 GET /upb/ (130.199.254.68) 0.00ms
404 GET /vcard/ (130.199.254.68) 1.00ms
404 GET /vcards/ (130.199.254.68) 0.00ms
404 GET /vhcs2/ (130.199.254.68) 0.00ms
404 GET /vicidial/ (130.199.254.68) 0.00ms
404 GET /viewvc/ (130.199.254.68) 0.00ms
404 GET /viewvc.cgi/ (130.199.254.68) 0.00ms
404 GET /vtiger/ (130.199.254.68) 0.00ms
404 GET /wb/ (130.199.254.68) 0.00ms
404 GET /wbboard/ (130.199.254.68) 0.00ms
404 GET /webftp/ (130.199.254.68) 0.00ms
